In [1]:
!pip -q install transformers accelerate bitsandbytes trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 28.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 98.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.2/562.2 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 74.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211

In [2]:
# Import necessary packages for the fine-tuning process
import os                          # Operating system functionalities
import torch                       # PyTorch library for deep learning
from datasets import load_dataset  # Loading datasets for training
from transformers import (
    AutoModelForCausalLM,          # AutoModel for language modeling tasks
    AutoTokenizer,                # AutoTokenizer for tokenization
    BitsAndBytesConfig,           # Configuration for BitsAndBytes
    HfArgumentParser,             # Argument parser for Hugging Face models
    TrainingArguments,            # Training arguments for model training
    pipeline,                     # Creating pipelines for model inference
    logging,                      # Logging information during training
)
from peft import LoraConfig, PeftModel,PeftConfig
from trl import SFTTrainer

from datasets import load_dataset, Dataset

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from google.colab import files
import os
import logging as log
import sqlite3
from typing import Any, Dict, Iterator, List, Mapping, Optional
import re
from pydantic import BaseModel, Field
import json
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

2025-09-16 05:34:43.817702: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758000884.145867      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758000884.228681      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Utils

In [3]:
def setup_logging():
    # Remove all handlers associated with the root logger object.
    for handler in log.root.handlers[:]:
        log.root.removeHandler(handler)

    log.basicConfig(
        level=log.WARNING,
        format="%(asctime)s [%(levelname)s] %(message)s",
        handlers=[log.StreamHandler()],
    )

In [4]:
def sql_process(example):
    create_statements = re.findall(r'(?:|\n|^)CREATE\s+TABLE\s+\w+\s*\([^;]+;', example, re.DOTALL)
    # Thêm IF NOT EXISTS cho từng câu lệnh
    create_statements = [
        re.sub(r'CREATE\s+TABLE', 'CREATE TABLE IF NOT EXISTS', stmt, flags=re.IGNORECASE)
        for stmt in create_statements
    ]
    
    schema = " ".join(create_statements)
    data = re.sub(r'CREATE\s+TABLE\s+\w+\s*\([^;]+;', '', example, flags=re.DOTALL).strip()
    return {"schema": schema, "data": data}

In [5]:
SYSTEM_PROMPT = (
    "You are a database agent. ALWAYS output EXACT JSON with keys: "
    '{"tool":"execute_sql","query":"<SQL>","meta":{"db":"<DB>"}}. '
    "Output only a single JSON object and nothing else — no explanation, no commentary."
)

def make_question(schema, question):
    system = SYSTEM_PROMPT + "\nConsider the following tables and their schemas:\n" + schema.strip() + "\n"
    user = question.strip()
    return {"system": system, "user": user}

In [6]:
def get_dataset():
    ds = load_dataset("gretelai/synthetic_text_to_sql", split="test")
    ds = ds.map(lambda x: sql_process(x["sql_context"]))
    ds = ds.map(lambda x: make_question(x['schema'], x['sql_prompt']))
    ds = ds.select_columns(['schema', 'data', 'sql', 'domain', 'sql_prompt'])
    return ds

In [7]:
logger = log.getLogger(__name__)
setup_logging()

# Dataset EDA

In [8]:
dataset = load_dataset("gretelai/synthetic_text_to_sql", split="test")
print(dataset[0])

README.md: 0.00B [00:00, ?B/s]

(…)nthetic_text_to_sql_train.snappy.parquet:   0%|          | 0.00/32.4M [00:00<?, ?B/s]

(…)ynthetic_text_to_sql_test.snappy.parquet:   0%|          | 0.00/1.90M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5851 [00:00<?, ? examples/s]

{'id': 1, 'domain': 'artificial intelligence', 'domain_description': 'AI data on algorithmic fairness, AI safety, explainable AI, and creative AI applications.', 'sql_complexity': 'basic SQL', 'sql_complexity_description': 'basic SQL with a simple select statement', 'sql_task_type': 'analytics and reporting', 'sql_task_type_description': 'generating reports, dashboards, and analytical insights', 'sql_prompt': "What is the average explainability score of creative AI applications in 'Europe' and 'North America' in the 'creative_ai' table?", 'sql_context': "CREATE TABLE creative_ai (application_id INT, name TEXT, region TEXT, explainability_score FLOAT); INSERT INTO creative_ai (application_id, name, region, explainability_score) VALUES (1, 'ApplicationX', 'Europe', 0.87), (2, 'ApplicationY', 'North America', 0.91), (3, 'ApplicationZ', 'Europe', 0.84), (4, 'ApplicationAA', 'North America', 0.93), (5, 'ApplicationAB', 'Europe', 0.89);", 'sql': "SELECT AVG(explainability_score) FROM creativ

In [9]:
device_map = {"": 0}

# Model evalutation

In [10]:
path = "/kaggle/input/model-check/merged_model"

# Step 4 :Load base model
model = AutoModelForCausalLM.from_pretrained(
    path,
    device_map=device_map,
    local_files_only=True
)

tokenizer = AutoTokenizer.from_pretrained(path, local_files_only=True)

In [11]:
data = get_dataset()
data[0]

Map:   0%|          | 0/5851 [00:00<?, ? examples/s]

Map:   0%|          | 0/5851 [00:00<?, ? examples/s]

{'schema': 'CREATE TABLE IF NOT EXISTS creative_ai (application_id INT, name TEXT, region TEXT, explainability_score FLOAT);',
 'data': "INSERT INTO creative_ai (application_id, name, region, explainability_score) VALUES (1, 'ApplicationX', 'Europe', 0.87), (2, 'ApplicationY', 'North America', 0.91), (3, 'ApplicationZ', 'Europe', 0.84), (4, 'ApplicationAA', 'North America', 0.93), (5, 'ApplicationAB', 'Europe', 0.89);",
 'sql': "SELECT AVG(explainability_score) FROM creative_ai WHERE region IN ('Europe', 'North America');",
 'domain': 'artificial intelligence',
 'sql_prompt': "What is the average explainability score of creative AI applications in 'Europe' and 'North America' in the 'creative_ai' table?"}

In [12]:
example = data[0]
schema = example['schema']
user = example['sql_prompt']

In [13]:
system = SYSTEM_PROMPT + "\nConsider the following tables and their schemas:\n" + schema.strip() + "\n"
messages = [{"role":"system","content":system}, {"role":"user","content":user}]
prompt_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [14]:
prompt_text

'<|system|>\nYou are a database agent. ALWAYS output EXACT JSON with keys: {"tool":"execute_sql","query":"<SQL>","meta":{"db":"<DB>"}}. Output only a single JSON object and nothing else — no explanation, no commentary.\nConsider the following tables and their schemas:\nCREATE TABLE IF NOT EXISTS creative_ai (application_id INT, name TEXT, region TEXT, explainability_score FLOAT);\n</s>\n<|user|>\nWhat is the average explainability score of creative AI applications in \'Europe\' and \'North America\' in the \'creative_ai\' table?</s>\n<|assistant|>\n'

In [15]:
inputs = tokenizer(prompt_text, return_tensors="pt").to(model.device)
input_ids = inputs["input_ids"]
input_ids

tensor([[    1, 32001, 29871,    13,  3492,   526,   263,  2566, 10823, 29889,
         14445, 12982, 21554,  1962,  8528, 17923,  4663,   411,  6611, 29901,
          8853, 10154,  4710,  7978, 29918,  2850,  3284,  1972,  4710, 29966,
          4176, 29958,  3284,  7299, 28819,  2585,  4710, 29966,  4051, 11903,
         27243, 10604,   871,   263,  2323,  4663,  1203,   322,  3078,  1683,
           813,   694,  8252, 29892,   694,  3440,   653, 29889,    13, 13696,
          1241,   278,  1494,  6131,   322,  1009,  1364,  8609, 29901,    13,
         27045, 10911, 10762,  6058, 28731,   907,  1230, 29918,  1794,   313,
          6214, 29918,   333, 19578, 29892,  1024,   323, 12194, 29892,  5120,
           323, 12194, 29892,  5649,  3097, 29918, 13628,   383,  3927,  1299,
           416,    13,   829, 29879, 29958,    13, 32002, 29871,    13,  5618,
           338,   278,  6588,  5649,  3097,  8158,   310,   907,  1230,   319,
         29902,  8324,   297,   525, 15654, 29915,  

In [18]:
gen = model.generate(**inputs, max_new_tokens=100, do_sample=False)
out = tokenizer.decode(gen[0])
out

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


'<s><|system|> \nYou are a database agent. ALWAYS output EXACT JSON with keys: {"tool":"execute_sql","query":"<SQL>","meta":{"db":"<DB>"}}. Output only a single JSON object and nothing else — no explanation, no commentary.\nConsider the following tables and their schemas:\nCREATE TABLE IF NOT EXISTS creative_ai (application_id INT, name TEXT, region TEXT, explainability_score FLOAT);\n</s>\n<|user|> \nWhat is the average explainability score of creative AI applications in \'Europe\' and \'North America\' in the \'creative_ai\' table?</s>\n<|assistant|> \n{"tool": "execute_sql", "query": "SELECT AVG(explainability_score) FROM creative_ai WHERE region IN (\'Europe\', \'North America\');", "meta": {"db": "artificial intelligence_db"}}</s><unk>߬</s><unk><unk><unk><unk><unk><unk> <unk><unk><unk><unk><unk><unk> <unk>     <unk><unk><unk><unk> <unk> <unk><unk> <unk><unk><unk><unk><unk><unk> <unk>'

In [42]:
def nl2sql_infer(model, tokenizer, schema_text, user_question, max_new_tokens=256):
    system = SYSTEM_PROMPT + "\nConsider the following tables and their schemas:\n" + schema_text.strip() + "\n"
    messages = [
        {"role":"system","content":system},
        {"role":"user","content":user_question}
    ]
    prompt_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    inputs = tokenizer(prompt_text, return_tensors="pt").to(model.device)

    gen = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)
    out = tokenizer.decode(gen[0], skip_special_tokens=False)

    # extract the first JSON object in output
    match = re.search(r"<\|assistant\|>\s*(\{.*?\})\s*</s>", out, re.DOTALL)
    if match:
        assistant_content = match.group(1)
        return assistant_content

In [43]:
answer = nl2sql_infer(model, tokenizer, schema, user, max_new_tokens=100)

if isinstance(answer, str):
    answer = json.loads(answer)
answer

KeyboardInterrupt: 

In [21]:
sqliteConnection = sqlite3.connect(f"{example['domain']}_db")
cursor = sqliteConnection.cursor()

create_query = data[0]['schema']
cursor.execute(create_query)

insert_query = data[0]['data']
cursor.execute(insert_query)

query = data[0]['sql']
cursor.execute(query)
result = cursor.fetchall()
print(result)

query = answer['query']
cursor.execute(query)
result = cursor.fetchall()
print(result)

[(0.8880000000000001,)]
[(0.8880000000000001,)]


In [46]:
limit = 100
data_ = data.select(range(limit))

In [47]:
import sqlite3, json
from datasets import Features, Value

debug_logs = [] 

def safe_evaluate_debug(record):
    try:
        schema = record.get("schema", "")
        user_prompt = record.get("sql_prompt", "")
        orig_sql = record.get("sql", "")
        data_sql = record.get("data", "")

        try:
            answer = nl2sql_infer(model, tokenizer, schema, user_prompt, max_new_tokens=100)
        except Exception as e:
            debug_logs.append({
                "orig_sql": orig_sql,
                "gen_query": None,
                "error": f"nl2sql_infer error: {e}",
                "correct": False
            })
            return {"correct": False, "other": True}

        if isinstance(answer, str):
            try:
                answer = json.loads(answer)
            except Exception:
                answer = {"query": answer}

        if not isinstance(answer, dict) or "query" not in answer:
            debug_logs.append({
                "orig_sql": orig_sql,
                "gen_query": str(answer),
                "error": "No query field",
                "correct": False
            })
            return {"correct": False, "other": True}

        gen_query = answer["query"]

        if not isinstance(gen_query, str) or gen_query.strip() == "":
            debug_logs.append({
                "orig_sql": orig_sql,
                "gen_query": None,
                "error": "Empty query",
                "correct": False
            })
            return {"correct": False, "other": True}

        if isinstance(orig_sql, str) and orig_sql.strip().upper().startswith(("DELETE", "UPDATE", "INSERT")):
            return {"correct": False, "other": True}

        with sqlite3.connect(":memory:") as conn:
            cur = conn.cursor()
            try:
                if schema:
                    cur.executescript(schema)
                if data_sql:
                    cur.executescript(data_sql)
            except Exception as e:
                debug_logs.append({
                    "orig_sql": orig_sql,
                    "gen_query": gen_query,
                    "error": f"Schema/data error: {e}",
                    "correct": False
                })
                return {"correct": False, "other": True}

            try:
                cur.execute(orig_sql)
                orig_result = cur.fetchall()
            except Exception as e:
                orig_result = None
                debug_logs.append({
                    "orig_sql": orig_sql,
                    "gen_query": gen_query,
                    "error": f"Orig SQL error: {e}",
                    "correct": False
                })

            try:
                cur.execute(gen_query)
                gen_result = cur.fetchall()
                error = None
            except Exception as e:
                gen_result = None
                error = f"Gen SQL error: {e}"

        correct = (orig_result == gen_result)
        debug_logs.append({
            "orig_sql": orig_sql,
            "gen_query": gen_query,
            "error": error,
            "correct": bool(correct)
        })

        return {"correct": bool(correct), "other": False}

    except Exception as e:
        debug_logs.append({
            "orig_sql": record.get("sql", ""),
            "gen_query": None,
            "error": f"safe_evaluate exception: {e}",
            "correct": False
        })
        return {"correct": False, "other": True}


features = Features({"correct": Value("bool"), "other": Value("bool")})

results = data_.map(
    safe_evaluate_debug,
    num_proc=1,
    remove_columns=data_.column_names,
    features=features,
    load_from_cache_file=False
)

# Tính accuracy
correct_count = sum(results["correct"])
accuracy = correct_count / len(results) if len(results) > 0 else 0.0
print("Accuracy:", accuracy)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Accuracy: 0.41


In [48]:
for log in debug_logs[:5]:
    print(json.dumps(log, indent=2))

{
  "orig_sql": "SELECT AVG(explainability_score) FROM creative_ai WHERE region IN ('Europe', 'North America');",
  "gen_query": "SELECT AVG(explainability_score) FROM creative_ai WHERE region IN ('Europe', 'North America');",
  "error": null,
  "correct": true
}
{
  "orig_sql": "SELECT launch_provider, COUNT(*) FROM Accidents GROUP BY launch_provider;",
  "gen_query": "SELECT COUNT(*) FROM Accidents WHERE launch_provider IN ('SpaceX', 'Blue Origin');",
  "error": null,
  "correct": false
}
{
  "orig_sql": "SELECT MAX(quantity) FROM sales;",
  "gen_query": "SELECT MAX(quantity) FROM sales WHERE location = 'Seafood';",
  "error": null,
  "correct": false
}
{
  "orig_sql": "SELECT SUM(budget) FROM Movies_Release_Year WHERE release_year < 2010;",
  "gen_query": "SELECT SUM(budget) FROM Movies_Release_Year WHERE release_year < 2010;",
  "error": null,
  "correct": true
}
{
  "orig_sql": "SELECT plant_name, SUM(co2_emission_per_ton_produced) AS total_co2_emission FROM co2_emissions GROUP BY